To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth your local device, follow [our guide](https://docs.unsloth.ai/get-started/install-and-update). This notebook is licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News


Introducing FP8 precision training for faster RL inference. [Read Blog](https://docs.unsloth.ai/new/fp8-reinforcement-learning).

Unsloth's [Docker image](https://hub.docker.com/r/unsloth/unsloth) is here! Start training with no setup & environment issues. [Read our Guide](https://docs.unsloth.ai/new/how-to-train-llms-with-unsloth-and-docker).

[gpt-oss RL](https://docs.unsloth.ai/new/gpt-oss-reinforcement-learning) is now supported with the fastest inference & lowest VRAM. Try our [new notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/gpt-oss-(20B)-GRPO.ipynb) which creates kernels!

Introducing [Vision](https://docs.unsloth.ai/new/vision-reinforcement-learning-vlm-rl) and [Standby](https://docs.unsloth.ai/basics/memory-efficient-rl) for RL! Train Qwen, Gemma etc. VLMs with GSPO - even faster with less VRAM.

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Unsloth

Goal: To convert `Qwen3-4B-Base` into a reasoning model via GRPO by using OpenR1's Math dataset.

We first pre fine-tune the model to make GRPO skip trying to match formatting - this speeds GRPO up.

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/root/autodl-tmp/unsloth/Qwen3-4B-Base",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # Reduce if out of memory
    local_files_only = True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
WARNING 01-15 14:24:46 [vllm.py:1403] Current vLLM config is not set.
INFO 01-15 14:24:46 [scheduler.py:230] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 01-15 14:24:46 [vllm_utils.py:702] Unsloth: Patching vLLM v1 graph capture
==((====))==  Unsloth 2026.1.2: Fast Qwen3 patching. Transformers: 4.57.3. vLLM: 0.13.0.
   \\   /|    NVIDIA GeForce RTX 4080 SUPER. Num GPUs = 1. Max memory: 31.473 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading /root/autodl-tmp/unsloth/Qwen3-4B-Base with actual GPU utilization = 49.54%
Unsloth: Your GPU has CU

/root/miniconda3/lib/python3.12/site-packages/pydantic/type_adapter.py:605: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `enum` - serialized value may not be as expected [field_name='mode', input_value=3, input_type=int])
  return self.serializer.to_python(


INFO 01-15 14:24:48 [scheduler.py:230] Chunked prefill is enabled with max_num_batched_tokens=4096.
INFO 01-15 14:24:48 [core.py:93] Initializing a V1 LLM engine (v0.13.0) with config: model='/root/autodl-tmp/unsloth/Qwen3-4B-Base', speculative_config=None, tokenizer='/root/autodl-tmp/unsloth/Qwen3-4B-Base', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, structured_outputs_config=StructuredOutputsConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_parser='', reasoning_parser_plugin='', enable_in_reasoning=False), observability_config=ObservabilityConfig(show_hidden_metrics_for_version=None, o

/root/miniconda3/lib/python3.12/site-packages/pydantic/type_adapter.py:605: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `enum` - serialized value may not be as expected [field_name='mode', input_value=3, input_type=int])
  return self.serializer.to_python(


INFO 01-15 14:24:48 [parallel_state.py:1203] world_size=1 rank=0 local_rank=0 distributed_init_method=tcp://172.17.0.11:36015 backend=nccl
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
INFO 01-15 14:24:49 [parallel_state.py:1411] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, PCP rank 0, TP rank 0, EP rank 0
INFO 01-15 14:24:49 [topk_topp_sampler.py:47] Using FlashInfer for top-p & top-k sampl

...

INFO 01-15 14:25:16 [core.py:259] init engine (profile, create kv cache, warmup model) took 23.65 seconds
INFO 01-15 14:25:17 [llm.py:360] Supported tasks: ('generate',)
Unsloth: Just some info: will skip parsing ['norm2', 'k_norm', 'post_attention_layernorm', 'layer_norm2', 'post_layernorm', 'layer_norm1', 'pre_feedforward_layernorm', 'input_layernorm', 'q_norm', 'attention_norm', 'ffn_norm', 'post_feedforward_layernorm', 'norm', 'norm1']


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of Qwen3ForCausalLM were not initialized from the model checkpoint at /root/autodl-tmp/unsloth/Qwen3-4B-Base and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Performing substitution for additional_keys=set()
Unsloth: Just some info: will skip parsing ['norm2', 'k_norm', 'post_attention_layernorm', 'layer_norm2', 'post_layernorm', 'cross_attn_input_layernorm', 'cross_attn_post_attention_layernorm', 'layer_norm1', 'pre_feedforward_layernorm', 'input_layernorm', 'q_norm', 'attention_norm', 'ffn_norm', 'post_feedforward_layernorm', 'norm', 'norm1']


Unsloth 2026.1.2 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


### GRPO chat template
Since we're using a base model, we should set a chat template. You can make your own chat template as well!
1. DeepSeek uses `<think>` and `</think>`, but this is **not** necessary - you can customize it however you like!
2. A `system_prompt` is recommended to at least guide the model's responses.

In [2]:
reasoning_start = "<start_working_out>" # Acts as <think>
reasoning_end   = "<end_working_out>"   # Acts as </think>
solution_start  = "<SOLUTION>"
solution_end    = "</SOLUTION>"

system_prompt = \
f"""You are given a problem.
Think about the problem and provide your working out.
Place it between {reasoning_start} and {reasoning_end}.
Then, provide your solution between {solution_start}{solution_end}"""
system_prompt

'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>'

We create a simple chat template below. Notice `add_generation_prompt` includes prepending `<start_working_out>` to guide the model to start its reasoning process.

In [3]:
chat_template = \
    "{% if messages[0]['role'] == 'system' %}"\
        "{{ messages[0]['content'] + eos_token }}"\
        "{% set loop_messages = messages[1:] %}"\
    "{% else %}"\
        "{{ '{system_prompt}' + eos_token }}"\
        "{% set loop_messages = messages %}"\
    "{% endif %}"\
    "{% for message in loop_messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ message['content'] }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ message['content'] + eos_token }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}{{ '{reasoning_start}' }}"\
    "{% endif %}"

# Replace with out specific template:
chat_template = chat_template\
    .replace("'{system_prompt}'",   f"'{system_prompt}'")\
    .replace("'{reasoning_start}'", f"'{reasoning_start}'")
tokenizer.chat_template = chat_template

Let's see how our chat template behaves on an example:

In [4]:
tokenizer.apply_chat_template([
    {"role" : "user", "content" : "What is 1+1?"},
    {"role" : "assistant", "content" : f"{reasoning_start}I think it's 2.{reasoning_end}{solution_start}2{solution_end}"},
    {"role" : "user", "content" : "What is 2+2?"},
], tokenize = False, add_generation_prompt = True)

"You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION><|endoftext|>What is 1+1?<start_working_out>I think it's 2.<end_working_out><SOLUTION>2</SOLUTION><|endoftext|>What is 2+2?<start_working_out>"

### Pre fine-tuning for formatting
We now use a subset of NVIDIA's [Open Math Reasoning dataset](https://huggingface.co/datasets/nvidia/OpenMathReasoning) which was filtered to only include high quality DeepSeek R1 traces.

We'll only filter ~59 or so examples to first "prime" / pre fine-tune the model to understand our custom GRPO formatting.

In [5]:
from datasets import load_dataset
import pandas as pd
import numpy as np

dataset = load_dataset("/root/autodl-tmp/datasets/OpenMathReasoning-mini", split = "cot")
dataset = dataset.to_pandas()[
    ["expected_answer", "problem", "generated_solution"]
]

# Try converting to number - if not, replace with NaN
is_number = pd.to_numeric(pd.Series(dataset["expected_answer"]), errors = "coerce").notnull()
# Select only numbers
dataset = dataset.iloc[np.where(is_number)[0]]

dataset

,expected_answer,problem,generated_solution
0,14,Given $\sqrt{x^2+165}-\sqrt{x^2-52}=7$ and $x$...,"<think>\nOkay, let's see. I need to solve the ..."
6,-2,Find the value of the parameter $a$ for which ...,"<think>\nOkay, so I need to find the value of ..."
9,18,What is the sum of all real numbers $x$ for wh...,"<think>\nOkay, so I need to solve the equation..."
13,2,Evaluate the sum \(\sum_{n=1}^\infty \frac{\ph...,"<think>\nOkay, so I need to evaluate the infin..."
17,30,What is the largest positive integer that divi...,"<think>\nAlright, so I need to find the larges..."
...,...,...,...
19243,244,"Let \( p \), \( q \), and \( r \) be the disti...","<think>\nOkay, so I need to find the value of ..."
19245,1,A bug is on the $0$ of a number line. At any p...,"<think>\nOkay, so I have this problem where a ..."
19247,4,A bus left point X for point Y. Two hours late...,"<think>\nOkay, let's tackle this problem step ..."
19248,18,Each interior angle of a regular n-gon measure...,"<think>\nOkay, let's see. I need to find the n..."


We have to format the dataset to follow our GRPO style formatting:

In [6]:
def format_dataset(x):
    expected_answer = x["expected_answer"]
    problem = x["problem"]

    # Remove generated <think> and </think>
    thoughts = x["generated_solution"]
    thoughts = thoughts.replace("<think>", "").replace("</think>", "")

    # Strip newlines on left and right
    thoughts = thoughts.strip()
    # Add our custom formatting
    final_prompt = \
        reasoning_start + thoughts + reasoning_end + \
        solution_start + expected_answer + solution_end
    return [
        {"role" : "system",    "content" : system_prompt},
        {"role" : "user",      "content" : problem},
        {"role" : "assistant", "content" : final_prompt},
    ]

dataset["Messages"] = dataset.apply(format_dataset, axis = 1)

Check to see if it worked:

In [7]:
tokenizer.apply_chat_template(dataset["Messages"][0], tokenize = False)

"You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION><|endoftext|>Given $\\sqrt{x^2+165}-\\sqrt{x^2-52}=7$ and $x$ is positive, find all possible values of $x$.<start_working_out>Okay, let's see. I need to solve the equation √(x² + 165) - √(x² - 52) = 7, and find all positive values of x. Hmm, radicals can be tricky, but maybe if I can eliminate the square roots by squaring both sides. Let me try that.\n\nFirst, let me write down the equation again to make sure I have it right:\n\n√(x² + 165) - √(x² - 52) = 7.\n\nOkay, so the idea is to isolate one of the radicals and then square both sides. Let me try moving the second radical to the other side:\n\n√(x² + 165) = 7 + √(x² - 52).\n\nNow, if I square both sides, maybe I can get rid of the square roots. Let's do that:\n\n(√(x² + 165))² = (7 + √(x² - 52))².\n\nSimplifying the left side:\n\nx² + 

Let's truncate the pre fine-tuning dataset to `max_seq_length/2` since we don't want too long reasoning traces.

Note this might take 2 minutes!

In [8]:
dataset["N"] = dataset["Messages"].apply(lambda x: len(tokenizer.apply_chat_template(x)))

dataset = dataset.loc[dataset["N"] <= max_seq_length/2].copy()
dataset.shape

(59, 5)

We then tokenize the messages and convert it to a Hugging Face compatible dataset format:

In [9]:
from datasets import Dataset

dataset["text"] = tokenizer.apply_chat_template(dataset["Messages"].values.tolist(), tokenize = False)
dataset = Dataset.from_pandas(dataset)
dataset

Dataset({
    features: ['expected_answer', 'problem', 'generated_solution', 'Messages', 'N', 'text', '__index_level_0__'],
    num_rows: 59
})

Let's now pre fine-tune the model so it follows our custom GRPO formatting!

In [10]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 20,
        num_train_epochs = 5, # Set this for 1 full training run.
        learning_rate = 5e-5, # Reduce to 2e-5 for long training runs
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

num_proc must be <= 59. Reducing num_proc to 59 for dataset of size 59.
[datasets.arrow_dataset|WARNING]num_proc must be <= 59. Reducing num_proc to 59 for dataset of size 59.


Unsloth: Tokenizing ["text"] (num_proc=59):   0%|          | 0/59 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


In [11]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 59 | Num Epochs = 5 | Total steps = 40
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 66,060,288 of 4,088,528,384 (1.62% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,0.719100
20,0.596200
30,0.458100
40,0.389400


TrainOutput(global_step=40, training_loss=0.5406890094280243, metrics={'train_runtime': 111.5212, 'train_samples_per_second': 2.645, 'train_steps_per_second': 0.359, 'total_flos': 5935482689018880.0, 'train_loss': 0.5406890094280243, 'epoch': 5.0})

Let's check if the model has learnt to follow the custom format:

In [12]:
text = tokenizer.apply_chat_template(
    dataset[0]["Messages"][:2],
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    temperature = 0,
    max_new_tokens = 1024,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
Then, provide your solution between <SOLUTION></SOLUTION><|endoftext|>Jenifer has 82 cents in pennies and nickels. Her younger brother mistook all her nickels for dimes and counted the total as $1.47. How many pennies does Jenifer have?<start_working_out>Okay, let's see. Jenifer has 82 cents in pennies and nickels. Her brother thought all the nickels were dimes and counted the total as $1.47. Hmm, I need to find out how many pennies she has. Let me break this down.

First, let's define the variables. Let's say the number of nickels Jenifer has is N, and the number of pennies is P. Since nickels are worth 5 cents and pennies are worth 1 cent, the total amount of money she has is:

5N + P = 82 cents.

Now, her brother thought all the nickels were dimes. Dimes are worth 10 cents. So if he counted all the nickels as dimes, the total would be:

10N + P =

Yes it did follow the formatting! Great! Let's remove some items before the GRPO step

In [13]:
del dataset
torch.cuda.empty_cache()
import gc
gc.collect()

0

### Data Prep
<a name="Data"></a>

We're using Hugging Face's [Open R1 Math dataset](https://huggingface.co/datasets/open-r1/DAPO-Math-17k-Processed). You can also utilize OpenAI's famous [GSM8K dataset](https://huggingface.co/datasets/openai/gsm8k)

In [14]:
from datasets import load_dataset
dataset = load_dataset("/root/autodl-tmp/datasets/DAPO-Math-17k-Processed", "en", split = "train")
dataset

Dataset({
    features: ['prompt', 'solution', 'data_source', 'source_prompt', 'ability', 'reward_model', 'extra_info'],
    num_rows: 14116
})

Let's look at the first row:

In [15]:
dataset[0]["prompt"]

'In triangle $ABC$, $\\sin \\angle A = \\frac{4}{5}$ and $\\angle A < 90^\\circ$. Let $D$ be a point outside triangle $ABC$ such that $\\angle BAD = \\angle DAC$ and $\\angle BDC = 90^\\circ$. Suppose that $AD = 1$ and that $\\frac{BD}{CD} = \\frac{3}{2}$. If $AB + AC$ can be expressed in the form $\\frac{a\\sqrt{b}}{c}$ where $a, b, c$ are pairwise relatively prime integers, find $a + b + c$.'

In [16]:
dataset[0]["solution"]

'34'

In GSM8K, ee notice all answers like about have a ####, so we extract it. But for the Open R1 dataset, we can skip the below.

In [17]:
def extract_hash_answer(text):
    # if "####" not in text: return None
    # return text.split("####")[1].strip()
    return text
extract_hash_answer(dataset[0]["solution"])

'34'

Let's map the dataset! and see the first row:

In [18]:
dataset = dataset.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": x["prompt"]},
    ],
    "answer": extract_hash_answer(x["solution"]),
})
dataset[0]

{'prompt': [{'content': 'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>',
   'role': 'system'},
  {'content': 'In triangle $ABC$, $\\sin \\angle A = \\frac{4}{5}$ and $\\angle A < 90^\\circ$. Let $D$ be a point outside triangle $ABC$ such that $\\angle BAD = \\angle DAC$ and $\\angle BDC = 90^\\circ$. Suppose that $AD = 1$ and that $\\frac{BD}{CD} = \\frac{3}{2}$. If $AB + AC$ can be expressed in the form $\\frac{a\\sqrt{b}}{c}$ where $a, b, c$ are pairwise relatively prime integers, find $a + b + c$.',
   'role': 'user'}],
 'solution': '34',
 'data_source': 'math_dapo',
 'source_prompt': [{'content': 'Solve the following math problem step by step. The last line of your response should be of the form Answer: $Answer (without quotes) where $Answer is the answer to the problem.\n\nIn triangle $ABC$, $\\sin \\angle A = \\frac{4}{5}$ and $

We create a regex format to match the reasoning sections and answers:

In [19]:
import re

# Add optional EOS token matching
solution_end_regex = r"</SOLUTION>[\s]{0,}" + \
    "(?:" + re.escape(tokenizer.eos_token) + ")?"

match_format = re.compile(
    rf"{reasoning_end}.*?"\
    rf"{solution_start}(.+?){solution_end_regex}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)
match_format

re.compile(r'<end_working_out>.*?<SOLUTION>(.+?)</SOLUTION>[\s]{0,}(?:<\|endoftext\|>)?[\s]{0,}$',
           re.MULTILINE|re.DOTALL|re.UNICODE)

We verify it works:

In [20]:
match_format.findall(
    "Let me think!<end_working_out>"\
    f"<SOLUTION>\n2\n</SOLUTION>",
)

['\n2\n']

In [21]:
match_format.findall(
    "<start_working_out>Let me think!<end_working_out>"\
    f"<SOLUTION>  2  </SOLUTION>\n\n",
)

['  2  ']

We now want to create a reward function to match the format exactly - we reward it with 3 points if it succeeds:

In [22]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_format.search(response) is not None: score += 3.0
        scores.append(score)
    return scores

If it fails, we want to reward the model if it at least follows the format partially, by counting each symbol:

In [23]:
def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Count how many keywords are seen - we penalize if too many!
        # If we see 1, then plus some points!

        # No need to reward <start_working_out> since we always prepend it!
        # score += 0.5 if response.count(reasoning_start) == 1 else -1.0
        score += 0.5 if response.count(reasoning_end)   == 1 else -1.0
        score += 0.5 if response.count(solution_start)  == 1 else -1.0
        score += 0.5 if response.count(solution_end)    == 1 else -1.0
        scores.append(score)
    return scores

Finally, we want to extract the generated answer, and reward or penalize it! We also reward it based on how close the answer is to the true one via ratios:

In [24]:
def check_answer(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_format.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(-2.0)
            continue
        # Correct answer gets 5 points!
        if guess == true_answer:
            score += 5.0
        # Match if spaces are seen, but less reward
        elif guess.strip() == true_answer.strip():
            score += 3.5
        else:
            # We also reward it if the answer is close via ratios!
            # Ie if the answer is within some range, reward it!
            try:
                ratio = float(guess) / float(true_answer)
                if   ratio >= 0.9 and ratio <= 1.1: score += 2.0
                elif ratio >= 0.8 and ratio <= 1.2: score += 1.5
                else: score -= 2.5 # Penalize wrong answers
            except:
                score -= 4.5 # Penalize
        scores.append(score)
    return scores

Also sometimes it might not be 1 number as the answer, but like a sentence for example "The solution is $20" -> we extract 20.

We also remove possible commas for example as in 123,456

In [25]:
match_numbers = re.compile(
    solution_start + r".*?[\s]{0,}([-]?[\d\.\,]{1,})",
    flags = re.MULTILINE | re.DOTALL
)
print(match_numbers.findall("<SOLUTION>  0.34  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>  123,456  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>  -0.234  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>17</SOLUTION>"))

['0.34']
['123,456']
['-0.234']
['17']


We now prepare our main function which will print out the generated responses and the true answer, along with another reward function which converts text to float via `float` and sees if it's the same.

In [26]:
global PRINTED_TIMES
PRINTED_TIMES = 0
global PRINT_EVERY_STEPS
PRINT_EVERY_STEPS = 5

def check_numbers(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_numbers.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    # Print only every few steps
    global PRINTED_TIMES
    global PRINT_EVERY_STEPS
    if PRINTED_TIMES % PRINT_EVERY_STEPS == 0:
        print(
            '*'*20 + f"Question:\n{question}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}"
        )
    PRINTED_TIMES += 1

    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(-2.5)
            continue
        # Convert to numbers
        try:
            true_answer = float(true_answer.strip())
            # Remove commas like in 123,456
            guess       = float(guess.strip().replace(",", ""))
            scores.append(3.5 if guess == true_answer else -1.5)
        except:
            scores.append(0)
            continue
    return scores

Get the top 90% prompt length so we don't accidentally truncate them!

Ie we'll remove the top 10% long prompts.

In [27]:
tokenized = dataset.map(
    lambda x: {"tokens" : tokenizer.apply_chat_template(x["prompt"], add_generation_prompt = True, tokenize = True)},
    batched = True,
)
print(tokenizer.decode(tokenized[0]["tokens"]))
tokenized = tokenized.map(lambda x: {"L" : len(x["tokens"])})

import numpy as np
maximum_length = int(np.quantile(tokenized["L"], 0.9))
print("Max Length = ", maximum_length)

# Filter only samples smaller than 90% max length
dataset = dataset.select(np.where(np.array(tokenized["L"]) <= maximum_length)[0])
del tokenized

You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
Then, provide your solution between <SOLUTION></SOLUTION><|endoftext|>In triangle $ABC$, $\sin \angle A = \frac{4}{5}$ and $\angle A < 90^\circ$. Let $D$ be a point outside triangle $ABC$ such that $\angle BAD = \angle DAC$ and $\angle BDC = 90^\circ$. Suppose that $AD = 1$ and that $\frac{BD}{CD} = \frac{3}{2}$. If $AB + AC$ can be expressed in the form $\frac{a\sqrt{b}}{c}$ where $a, b, c$ are pairwise relatively prime integers, find $a + b + c$.<start_working_out>
Max Length =  201


<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [28]:
import gc
import torch

# 1. 删掉可能残留的 trainer 和 optimizer 对象
try:
    del trainer
except:
    pass

try:
    # 释放显存缓存
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
except:
    pass

# 2. 检查显存（看 Memory-Usage 是否降下来了）
!nvidia-smi

Thu Jan 15 14:28:43 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.105.08             Driver Version: 580.105.08     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4080 ...    On  |   00000000:4A:00.0 Off |                  N/A |
| 91%   43C    P2             87W /  320W |   17535MiB /  32760MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [29]:
max_prompt_length = maximum_length + 1 # + 1 just in case!
max_completion_length = max_seq_length - max_prompt_length

from vllm import SamplingParams
vllm_sampling_params = SamplingParams(
    min_p = 0.1,
    top_p = 1.0,
    top_k = -1,
    seed = 3407,
    stop = [tokenizer.eos_token],
    include_stop_str_in_output = True,
)

import os

# --- WandB settings ---

# 设置项目名称（会自动在 WandB 创建这个项目）
os.environ["WANDB_PROJECT"] = "unsloth-qwen3-grpo" 

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    vllm_sampling_params = vllm_sampling_params,
    temperature = 1.0,
    learning_rate = 5e-6,
    weight_decay = 0.001,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 300,
    save_steps = 100,
    report_to = "wandb", # Can use Weights & Biases
    output_dir = "outputs",

    # For optional training + evaluation
    # fp16_full_eval = True,
    # per_device_eval_batch_size = 4,
    # eval_accumulation_steps = 1,
    # eval_strategy = "steps",
    # eval_steps = 1,
)

And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [30]:
# For optional training + evaluation
# new_dataset = dataset.train_test_split(test_size = 0.01)

trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        match_format_approximately,
        check_answer,
        check_numbers,
    ],
    args = training_args,
    train_dataset = dataset,

    # For optional training + evaluation
    # train_dataset = new_dataset["train"],
    # eval_dataset = new_dataset["test"],
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 12,709 | Num Epochs = 1 | Total steps = 300
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 66,060,288 of 4,088,528,384 (1.62% trained)
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /root/.netrc.
wandb: Currently logged in as: 1425740715 (1425740715-beihang-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


WARNING 01-15 14:28:49 [input_processor.py:250] vLLM has deprecated support for supporting different tokenizers for different LoRAs. By default, vLLM uses base model's tokenizer. If you are using a LoRA with its own tokenizer, consider specifying `--tokenizer [lora_path]` to use the LoRA tokenizer.
Unsloth: Will smartly offload gradients to save VRAM!
********************Question:
Compute the number of positive integers that divide at least two of the integers in the set $\{1^1,2^2,3^3,4^4,5^5,6^6,7^7,8^8,9^9,10^{10}\}$. 
Answer:
22 
Response:
Okay, so I need to find out how many positive integers divide at least two of the numbers in this set. Let's see, the set is {1^1, 2^2, 3^3, ..., 10^10}. Hmm, that's quite a few numbers here. Maybe I should start by calculating the prime factorization of each of these. That might help identify the common divisors.

Let me list them first:

1. 1^1 = 1
2. 2^2 = 4
3. 3^3 = 27
4. 4^4 = 16^2 = (2^4)^2 = 2^8
5. 5^5 = 3125
6. 6^6 = (2*3)^6 = 2^6 * 3^6
7

Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,sampling / sampling_logp_difference / mean,sampling / sampling_logp_difference / max,sampling / importance_sampling_ratio / min,sampling / importance_sampling_ratio / mean,sampling / importance_sampling_ratio / max,kl,rewards / match_format_exactly / mean,rewards / match_format_exactly / std,rewards / match_format_approximately / mean,rewards / match_format_approximately / std,rewards / check_answer / mean,rewards / check_answer / std,rewards / check_numbers / mean,rewards / check_numbers / std
1,0.000000,-3.937500,4.840691,1694.125000,1220.000000,1846.000000,0.750000,1238.500000,1220.000000,1257.000000,0,0,0,0,0,0.089017,0.750000,1.388730,-1.875000,2.083095,-1.187500,2.506242,-1.625000,2.100170
2,0.000000,-4.875000,4.164854,1679.125000,1280.000000,1846.000000,0.625000,1401.000000,1280.000000,1570.000000,No Log,No Log,No Log,No Log,No Log,0.083957,0.375000,1.060660,-1.687500,1.869635,-2.062500,0.176777,-1.500000,2.070197
3,0.000100,2.625000,3.818729,1353.500000,810.000000,1846.000000,0.250000,1189.333374,810.000000,1667.000000,No Log,No Log,No Log,No Log,No Log,0.102044,1.125000,1.552648,0.000000,1.603567,0.625000,3.622844,0.875000,2.825269
4,0.000500,-0.437500,6.436473,1287.375000,639.000000,1846.000000,0.250000,1101.166748,639.000000,1601.000000,No Log,No Log,No Log,No Log,No Log,0.101941,1.875000,1.552648,0.187500,2.034304,-1.375000,2.587746,-1.125000,1.922610
5,-0.000200,-4.125000,5.508540,1570.250000,768.000000,1846.000000,0.625000,1110.666748,768.000000,1339.000000,No Log,No Log,No Log,No Log,No Log,0.073692,0.375000,1.060660,-1.875000,1.747447,-1.125000,2.474874,-1.500000,2.070197
6,0.000100,0.875000,6.622767,1336.625000,1024.000000,1846.000000,0.250000,1166.833374,1024.000000,1373.000000,No Log,No Log,No Log,No Log,No Log,0.117073,1.500000,1.603567,-0.375000,1.922610,-0.375000,3.324691,0.125000,2.825269
7,-0.000100,-3.937500,5.468380,1626.000000,521.000000,1846.000000,0.625000,1259.333374,521.000000,1764.000000,No Log,No Log,No Log,No Log,No Log,0.090329,0.375000,1.060660,-1.687500,2.034304,-1.125000,2.474874,-1.500000,2.070197
8,0.000000,-1.250000,4.970748,1664.750000,907.000000,1846.000000,0.625000,1362.666748,907.000000,1825.000000,No Log,No Log,No Log,No Log,No Log,0.081989,0.750000,1.388730,-1.500000,2.121320,-0.250000,3.240370,-0.250000,3.105295
9,0.000500,-4.437500,6.125000,1721.875000,1214.000000,1846.000000,0.625000,1515.000000,1214.000000,1678.000000,No Log,No Log,No Log,No Log,No Log,0.081035,0.375000,1.060660,-2.062500,1.781603,-1.125000,2.474874,-1.625000,2.100170
10,-0.000300,-5.500000,4.000000,1756.000000,1408.000000,1846.000000,0.750000,1486.000000,1408.000000,1564.000000,No Log,No Log,No Log,No Log,No Log,0.105136,0.375000,1.060660,-2.062500,1.781603,-1.562500,1.237437,-2.250000,0.462910


********************Question:
Letters $A, B, C,$ and $D$ represent four different digits from 0, 1, 2, 3, ..., 9. If $\frac{A+B}{C+D}$ is an integer that is as large as possible, what is the value of $A+B$? 
Answer:
17 
Response:
Okay, so I need to maximize the value of \(\frac{A+B}{C+D}\), where \(A, B, C,\) and \(D\) are different digits from 0 to 9. Let's think about this step by step.

First, to make a fraction as large as possible, the numerator should be as large as possible and the denominator as small as possible. But since all digits are different, I can't just pick the highest numbers without considering the other variables. Hmm.

Let me start by trying the highest possible sum for the numerator. The maximum sum of two different digits is 9 + 8 = 17. But then, I need to make the denominator as small as possible. The smallest two different digits are 0 and 1, which would make the denominator 1. So the fraction would be 17/1 = 17, which is the maximum possible given the constra

profiling/Time taken: UnslothGRPOTrainer._calculate_rewards,▅▂▁▂▁▂▂▂▁▂▁▁▂▆█▂▁▂▃▁▂▂▂▁▂▃▂▂▁▂▆▁▇▂▂▁▂▃▂▂
profiling/Time taken: UnslothGRPOTrainer._prepare_inputs,▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▆▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█▁▁▆▁
profiling/Time taken: UnslothGRPOTrainer.check_answer,▂▁▅▂▂▁▂▂▂▁▁▁▁▁▁▁▁▃▁▁▄█▁▂▁▂▁▁▁▂▂▁▁▁▁▂▂▁▁▁
profiling/Time taken: UnslothGRPOTrainer.check_numbers,▂▆▁▁▁▁▁▁▁▁▁▁▆▅▇▁▁▁▁▁▇▁▁▁▅▁▁▁▁▇▁▇▁█▅▆▁▁▁▅
profiling/Time taken: UnslothGRPOTrainer.match_format_approximately,█▄▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
profiling/Time taken: UnslothGRPOTrainer.match_format_exactly,▁▁▂▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
profiling/Time taken: UnslothGRPOTrainer.vLLM.generate,████▇██▇▇▂▃▇▆▂▃█▇▃▅▃▅▇▂▅▂▇▅▁▅▃██▄▅▅▄▇▇▃▂
train/completion_length,▅▇▇█▅▇▇▇▃▃▄▃▂▃▂▅▃▄▄▁▃▄▄▅▂▃▃▄▃▁▄▃▃▆▅▅▃▂▂▂
train/completions/clipped_ratio,▃█▄▅▅▄▆▅▂▁▁▁▁▁▂▁▁▁▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▄▁▁▁▂
train/completions/max_length,████████▃█▄▄█▃▇▂▄▅██▄███▁▂▆▅██▆▄▂▆▃█▄█▃▂
+23,...


TrainOutput(global_step=300, training_loss=7.928396663677025e-05, metrics={'train_runtime': 9370.1019, 'train_samples_per_second': 0.256, 'train_steps_per_second': 0.032, 'total_flos': 0.0, 'train_loss': 7.928396663677025e-05})

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [31]:
text = "What is the sqrt of 101?"

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

' - Quora\nWhat is the sqrt of 101?\n21 Related\nSort\nRecommended\nKamal D Borkar\n, B.E. (C) from K. N. Khetan College of Engineering (1962)\nAnswered 11 months ago\n10.0498756211208871011110110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [32]:
model.save_lora("grpo_saved_lora")

Verify LoRA is actually trained!

In [33]:
from safetensors import safe_open

tensors = {}
with safe_open("grpo_saved_lora/adapter_model.safetensors", framework = "pt") as f:
    # Verify both A and B are non zero
    for key in f.keys():
        tensor = f.get_tensor(key)
        n_zeros = (tensor == 0).sum() / tensor.numel()
        assert(n_zeros.item() != tensor.numel())

Now we load the LoRA and test:

In [34]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "What is the sqrt of 101?"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 2048,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

" To find the square root of 101, we can use a method that involves finding the integer part of the square root and then using a more precise method to get a more accurate value. However, for the purpose of this problem, we can use a simple estimation and a more accurate method to get a good approximation.\n\n1. **Estimate the integer part:**\n   - We know that \\(10^2 = 100\\) and \\(11^2 = 121\\). Since 101 is between 100 and 121, the integer part of the square root of 101 is 10. So, the square root of 101 is between 10 and 11.\n\n2. **Refine the estimate:**\n   - To get a more accurate value, we can use a method like the Babylonian method (also known as Heron's method) or a simple trial and error with decimal values. However, for this problem, we can use a more precise method to get a good approximation.\n\n3. **Use a calculator for a more accurate value:**\n   - Using a calculator, we find that the square root of 101 is approximately 10.04987562112089. Rounding to a reasonable numb

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# ----- network_turbo -----
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value
# ----- network_turbo -----

# Merge to 16bit
model.save_pretrained_merged("model/qwen3-4b-grpo-lora-merged-16bit/", tokenizer, save_method = "merged_16bit",)
model.push_to_hub_merged("surtrj/qwen3-4b-grpo-lora-merged-16bit", tokenizer, save_method = "merged_16bit", token = "")

Detected local model directory: /root/autodl-tmp/unsloth/Qwen3-4B-Base
No existing and accessible Hugging Face cache directory found.


Unsloth: Preparing safetensor model files:   0%|          | 0/2 [00:00<?, ?it/s]

Unsloth: Preparing safetensor model files:  50%|█████     | 1/2 [00:02<00:02,  2.45s/it]
Copied model-00001-of-00002.safetensors from local model directory


Unsloth: Preparing safetensor model files: 100%|██████████| 2/2 [00:05<00:00,  2.70s/it]
Copied model-00002-of-00002.safetensors from local model directory


Unsloth: Merging weights into 16bit:   0%|          | 0/2 [00:00<?, ?it/s]

Unsloth: Merging weights into 16bit:  50%|█████     | 1/2 [00:13<00:13, 13.06s/it]

Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [00:21<00:00, 10.87s/it]
Unsloth: Merge process complete. Saved to `/root/autodl-tmp/model/qwen3-4b-grpo-lora-merged-16bit`
Detected local model directory: /root/autodl-tmp/unsloth/Qwen3-4B-Base
Processing Files (1 / 1): 100%
 11.4MB / 11.4MB,  0.00B/s  
New Data Upload: 
  0.00B /  0.00B,  0.00B/s  
No existing and accessible Hugging Face cache directory found.


Unsloth: Preparing safetensor model files:   0%|          | 0/2 [00:00<?, ?it/s]

Unsloth: Preparing safetensor model files:  50%|█████     | 1/2 [00:02<00:02,  2.50s/it]
Copied model-00001-of-00002.safetensors from local model directory


Unsloth: Preparing safetensor model files: 100%|██████████| 2/2 [00:05<00:00,  2.95s/it]
Copied model-00002-of-00002.safetensors from local model directory


Unsloth: Merging weights into 16bit:   0%|          | 0/2 [00:00<?, ?it/s]
Processing Files (1 / 1): 100%
 4.97GB / 4.97GB,  260kB/s  
New Data Upload: 100%
 3.18GB / 3.18GB,  260kB/s  


Unsloth: Merging weights into 16bit:  50%|█████     | 1/2 [02:44<02:44, 164.07s/it]
Processing Files (1 / 1): 100%
 3.08GB / 3.08GB,  246kB/s  
New Data Upload: 100%
 3.08GB / 3.08GB,  246kB/s  


Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [05:34<00:00, 167.10s/it]
Unsloth: Merge process complete. Saved to `/root/autodl-tmp/surtrj/qwen3-4b-grpo-lora-merged-16bit`

In [40]:
# Just LoRA adapters
model.save_pretrained("model/qwen3-4b-grpo-lora/")
tokenizer.save_pretrained("model/qwen3-4b-grpo-lora/")

model.push_to_hub("surtrj/qwen3-4b-grpo-lora", token = "")
tokenizer.push_to_hub("surtrj/qwen3-4b-grpo-lora", token = "")


README.md:   0%|          | 0.00/573 [00:00<?, ?B/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Saved model to https://huggingface.co/surtrj/qwen3-4b-grpo-lora


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            